# 特征
绿色动力，2021/9/22 15时，周线级别5，10，20多头；日线级别10，20，30，60（均大于120， 250）；30分钟级别出现5，10，20，60，120多头；且出现放量涨，缩量跌，股价在30分钟10周期线上。

# 初始化和公共库

In [44]:
# unset proxy
! export HTTPS_PROXY=""

In [38]:
import jqdatasdk as jq
import os
import arrow
import datetime
import numpy as np
from jqdatasdk import *
import pandas as pd

account = os.getenv("JQ_ACCOUNT")
password = os.getenv("JQ_PASSWORD")

jq.auth(account, password)
valuations=jq.get_fundamentals(query(valuation))
secs = jq.get_all_securities()

def get_name(code):
    return secs[secs.index==code].iloc[0]["display_name"]

def get_valuation(code):
    return int(valuations[valuations.code==code].iloc[0]["market_cap"])

def choose_stocks(exclude_st=True, exclude_688=True, valuation_range=(100,2000)):
    result = secs
    if exclude_688:
        result = result[result.index.str.startswith("688")==False]
    if exclude_st:
        result = result[result.display_name.str.find("ST") == -1]
        
    codes = []
    for code in result.index:
        try:
            if valuation_range[0] <= get_valuation(code) <= valuation_range[1]:
                codes.append(code)
        except Exception:
            pass
            
    return codes

def top_n_argpos(ts: np.array, n: int) -> np.array:
    """get top n (max->min) elements and return argpos which its value ordered in descent

    Example:
        >>> top_n_argpos([4, 3, 9, 8, 5, 2, 1, 0, 6, 7], 2)
        array([2, 3])
    Args:
        ts (np.array): [description]
        n (int): [description]

    Returns:
        np.array: [description]
    """
    return np.argsort(ts)[-n:][::-1]

def moving_average(ts: np.array, win: int):
    """计算时间序列ts在win窗口内的移动平均

    Example:

        >>> ts = np.arange(7)
        >>> moving_average(ts, 5)
        >>> array([2.0000, 3.0000, 4.0000])

    """

    return np.convolve(ts, np.ones(win) / win, "valid")

In [104]:
import arrow
import datetime
def scan(end:str=None):
    results = []
    if end is None:
        end = arrow.now().date()
        end = datetime.datetime(end.year, end.month, end.day, 15)
    else:
        end = arrow.get(end).naive
        
    fields = ["date", "open", "high", "low", "close", "volume"]
    for i, code in enumerate(choose_stocks()):
        name = get_name(code)
#         if (i+1) % 100 == 0:
#             print(f"processing {i}th stock")
        try:
            # 先看当日是否存在放量涨、缩量跌。这样9个bar就够了
            n = 9
            bars = jq.get_bars(code, n, unit="30m", end_dt = end, fields=fields, 
                               df=False, fq_ref_date=end, include_now=True)

            frame = bars["date"]
            if frame[-1] != end or len(bars) != n: # 停牌中或者其它
                continue

            volume = bars["volume"]
            flags = bars["close"] > bars["open"]

            vcr = volume[1:]/volume[:-1]
            pos1, pos2 = top_n_argpos(vcr, 2)
            v1, v2 = vcr[pos1], vcr[pos2]
                
            if (not flags[pos1 + 1]) or v1 < 4: # 量一不为阳线,或者没有放量
                continue
                
            if (not flags[pos2 + 1]) and v2 > v1/3.5:
                # 量一为阳线，但第二大量为阴，但没有缩量到量一的1/3.5以下
                continue
                

            # 再看30分钟级别是否为均线多头
            m = 120
            bars = jq.get_bars(code, m, unit="30m", end_dt=end, fields=fields,
                              df=False, fq_ref_date=end, include_now=True)
            
            if len(bars) != m:
                print(f"required bars min30 len for {code} is {m}, {len(bars)} actual ")
                continue
                
            close = bars["close"]
            ma10 = moving_average(close, 10)
            ma20 = moving_average(close, 20)
            ma60 = moving_average(close, 60)
            ma120 = moving_average(close, 120)
            
            if not (close[-1] >= ma10[-1] >= ma20[-1] >= ma60[-1] >= ma120[-1]):
                # 30分钟未形成多头
                continue
                
            # 再看日线是否多头
            l = 120
            bars = jq.get_bars(code, l, unit="1d", end_dt=end.date(), fields=fields,
                              df = False, fq_ref_date=end.date(), include_now=True)
            
            if len(bars) != l:
                print(f"required bars len for {name} is {l}, {len(bars)} actual ")
                continue
                
            close = bars["close"]
            ma10 = moving_average(close, 10)
            ma20 = moving_average(close, 20)
            ma30 = moving_average(close, 30)
            ma60 = moving_average(close, 60)
            ma120 = moving_average(close, 120)
            
            pcr_5 = round(close[-1] / close[-6] - 1, 2)
            pcr_10 = round(close[-1] / close[-11] -1, 2)
            pcr_20 = round(close[-1] / close[-21] -1, 2)
            pcr_60 = round(close[-1] / close[-61] -1, 2)
            
            if (not (close[-1] >= ma10[-1] >= ma20[-1] >= ma30[-1] >= ma60[-1])) or \
            (ma60[-1] < ma120[-1] * 0.95):
                # 日线未形成多头
                #print(f"{name} 日线未形成多头")
                continue
            
            # 再看周线是否多头
            k = 20
            bars = jq.get_bars(code, k, unit="1w", end_dt=end.date(), fields=fields,
                              df = False, fq_ref_date=end.date(), include_now=True)
            
            if len(bars) != k:
                print(f"required bars len for {name} is {k}, {len(bars)} actual ")
                continue
                
            close = bars["close"]
            ma5 = moving_average(close, 5)
            ma10 = moving_average(close, 10)
            ma20 = moving_average(close, 20)
            
            if (not (close[-1] >= ma5[-1] >= ma10[-1])) or (ma10[-1] < ma20[-1] * 0.95):
                # 周线未多头
                #print(f"{name} 未形成周线多头")
                continue
            
            results.append([name, code, close[-1], pcr_5, pcr_10, pcr_20, pcr_60])
        except Exception as e:
            print(e)
            
    print(f"totally {i} stocks checked")
    df = pd.DataFrame(results, columns=["股票", "代码", "现价", "5日", "10日", "20日", "60日"])
    return df

In [ ]:
df = scan("2021-09-28 15:00:00")
df.style.format({
        "5日": "{:.0%}",
        "10日": "{:.0%}",
        "20日": "{:.0%}",
        "60日": "{:.0%}"
    })

In [95]:
df = scan("2021-09-27 15:00:00")
df.style.format({
        "5日": "{:.0%}",
        "10日": "{:.0%}",
        "20日": "{:.0%}",
        "60日": "{:.0%}"
    })

index -1 is out of bounds for axis 0 with size 0
required bars len for 杭州热电 is 120, 61 actual 
totally 1281 stocks checked


,股票,代码,现价,5日,10日,20日,60日
0,温氏股份,300498.XSHE,13.79,0.06,0.01,0.02,-0.03
1,广州发展,600098.XSHG,8.14,0.06,0.00,0.20,0.29
2,绿色动力,601330.XSHG,11.12,0.14,0.15,0.24,0.28


In [105]:
df = scan("2021-09-24 15:00:00")
df.style.format({
        "5日": "{:.0%}",
        "10日": "{:.0%}",
        "20日": "{:.0%}",
        "60日": "{:.0%}"
    })

index -1 is out of bounds for axis 0 with size 0
index -1 is out of bounds for axis 0 with size 0
required bars len for 东鹏饮料 is 120, 83 actual 
totally 1281 stocks checked


,股票,代码,现价,5日,10日,20日,60日
0,广宇发展,000537.XSHE,15.130000,61%,160%,223%,204%
1,东方能源,000958.XSHE,5.230000,6%,7%,20%,27%
2,杭锅股份,002534.XSHE,24.330000,13%,5%,15%,59%
3,华能国际,600011.XSHG,8.050000,21%,13%,55%,97%
4,深高速,600548.XSHG,10.890000,18%,17%,22%,25%
5,申能股份,600642.XSHG,7.200000,6%,7%,14%,20%
6,嘉泽新能,601619.XSHG,5.690000,17%,7%,32%,46%
7,三峰环境,601827.XSHG,10.660000,15%,16%,18%,13%
